# Project plan

## SVG Animator blueprint

1. Read the SVG file.
2. Analyse and understand the elements (LLM)
3. Update the SVG file (XML format) with appropriate ids where required. (LLM)
4. Create a new SVG (XML) with the animation prompt. (LLM)
5. Update the SVG file (XML) and write


In [ ]:
#llm setup
import os
from langchain_google_genai import ChatGoogleGenerativeAI


#Setup the API key

#Setup the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)


In [68]:
# Read the SVG file
# Analyse and understand the elements (LLM)

from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader

def analyse_svg_elements(svg_path):
    """
    Reads the svg file as a text file then Analyse and understand the elements of the SVG file.

    Args:
        svg_path (str): The path to the svg file.

    Returns:
        svg_text(str): The text version of the svg file.
        analysis_report(str): A report of the elements in the svg file.
    """

    #load the svg file as text using langchain text loader
    loader = TextLoader(svg_path)
    svg_text = loader.load()

    # Create chat prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert at analyzing SVG files. 
        Describe the visual elements, shapes, colors, text, and overall design present in the SVG."""),
        ("human", """SVG Content:

        {svg_content}
        """)
            ])

    # Create chain
    chain = prompt | llm

    #invoke the chain
    response = chain.invoke({
        "svg_content": svg_text
        })
    
    return svg_text, response.content


In [287]:
path_svg = "/Users/vishwajeetsawant/Work/animater/images/airport.svg"
svg_text, response = analyse_svg_elements(path_svg)

In [245]:
from langchain_core.output_parsers import BaseOutputParser

# 1. Define your custom output parser
class SvgOutputParser(BaseOutputParser[str]):
    """
    Parses the LLM output to extract the SVG code block.
    """
    def parse(self, text: str) -> str:
        """Parses the input text and returns the SVG code."""
        # Use regex to find the SVG content. re.DOTALL makes '.' match newlines.
        match = re.search(r'<svg.*</svg>', text, re.DOTALL)
        if not match:
            raise ValueError("Failed to find SVG code in the LLM output.")
        # Return the matched SVG code
        return match.group(0).strip()

In [266]:
# Update the SVG file (XML format) with appropriate ids where required. (LLM)

def update_svg_with_ids(svg_txt, analysis_report):
    """
    Update the svg file with appropriate ids where required.

    Args:
        svg_txt (str): The text version of the svg file.
        analysis_report (str): A report of the elements in the svg file.

    Returns:
        updted_svg_text(str): The updated text version of the svg file.
    """

    #parse the svg text
    svg_text = svg_txt[0].page_content

    # Create chat prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert at updating SVG files. 
        Analyse the SVG content and update the file with appropriate ids where required based on the analysis report.
        output should only include the xml text which can be saved as a svg image"""),
        ("human", """SVG text:

        {svg_txt}
        """)
            ])

    # Instantiate the parser
    svg_parser = SvgOutputParser()

    # Create chain
    chain = prompt | llm | svg_parser

    #invoke the chain
    response = chain.invoke({
        "svg_txt": svg_text
        })

    return response

In [288]:
updated_svg_text = update_svg_with_ids(svg_text, response)
updated_svg_text

'<svg xmlns="http://www.w3.org/2000/svg" height="48px" viewBox="0 -960 960 960" width="48px" fill="#e3e3e3"><path id="icon-path" d="m249-409 78-205H184l-56 82H80l38-133-38-132h48l56 82h143l-78-205h60l143 205h132q21 0 36 14.5t15 35.5q0 21-15 36t-36 15H452L309-409h-60ZM652-40 509-245H377q-21 0-36-15t-15-36q0-21 15-35.5t36-14.5h132l143-205h60l-79 205h144l56-82h47l-38 132 38 133h-47l-56-82H633l79 205h-60Z"/></svg>'

In [268]:
# Create a new SVG (XML) with the animation prompt. (LLM)



def animate_svg(updated_svg_text, analysis_report, user_prompt):
    """
    Create a new SVG file with animation.

    Args:
        updated_svg_text (str): The text version of the svg file with updated ids.
        analysis_report (str): A report of the elements in the svg file.
        user_prompt (str): Animation prompt from the user

    Returns:
        animated_svg(str): The updated text version of the svg file with animation.
    """

    system_prompt_text = """
You are an expert SVG Animation Engineer. Your sole function is to take a static SVG, an analysis of its components, and a user's animation request, and then produce a new, animated SVG. You will use SVG's native SMIL animation technology to achieve this.

**Your Goal:** To intelligently inject SMIL animation tags (`<animate>`, `<animateTransform>`, etc.) into the provided `svg_text` to bring the user's vision to life, returning a single, self-contained, animated SVG file.

---

**INPUTS:**

1.  **`svg_text`**: A string containing the complete XML content of the original, static SVG.
2.  **`analysis_report`**: A structured description of the SVG, identifying key shapes, paths, groups, and their corresponding IDs. This is your map to the SVG's structure.
3.  **`user_prompt`**: A natural language instruction from the user describing the desired animation.

---

**YOUR PROCESS:**

1.  **Synthesize Information:** Carefully read the `user_prompt` and cross-reference it with the `analysis_report`. Your first step is to identify exactly which elements (e.g., `<path id="line1">`, `<circle id="dot">`) are the targets for animation.

2.  **Devise an Animation Plan:**
    * **Choose the Right Tool:** You MUST use SMIL. This ensures the SVG is self-contained. Do not use CSS or JavaScript.
        * Use `<animate>` for attributes like `d`, `points`, `fill`, `stroke-opacity`, `x`, `y`, `r`.
        * Use `<animateTransform>` for transformations like `translate`, `scale`, or `rotate`.
    * **Determine Attributes:** Based on the user's prompt (e.g., "make it move," "change color," "spin around"), determine the specific SVG attribute to animate (e.g., `transform`, `fill`, `d`).
    * **Define Keyframes:** Plan the animation's values. Use a `values` attribute with a semicolon-separated list for a multi-step animation (e.g., `values="0; 10; 0"`). The animation should start and end in the SVG's original state to ensure a seamless loop.
    * **Set Timing:**
        * **Duration (`dur`)**: Choose a reasonable duration, typically between `1.5s` and `3s`, for a pleasant viewing experience.
        * **Looping (`repeatCount`)**: Always set this to `"indefinite"` for a continuous animation.
        * **Staggering (`begin`)**: If animating multiple elements, apply small delays to each subsequent element (e.g., `begin="0s"`, `begin="0.2s"`, `begin="0.4s"`) to create a more dynamic and professional-looking effect.

3.  **Construct the Final SVG:**
    * Take the original `svg_text` as your base.
    * Carefully insert the planned `<animate.../>` tags as children of their target elements.
    * **Crucially, do not alter any other part of the SVG.** Preserve all existing elements, attributes, and formatting.

---

**OUTPUT REQUIREMENTS:**

* Your entire response MUST be the final, complete, and valid SVG/XML code.
* Enclose the code in a single markdown code block.
* **DO NOT** include any conversational text, explanations, or apologies before or after the code block. Your output is the code and nothing else.
"""

    # The human-readable message template that will contain the dynamic inputs.
    human_message_template = """
Here is the SVG and the animation request. Please generate the animated SVG.

**SVG Text:**
```xml
{svg_text}
```

**Analysis Report:**
```
{analysis_report}
```

**User Animation Prompt:**
```
{user_prompt}
```
"""


    # Create the ChatPromptTemplate from the system and human message strings.
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt_text),
        ("human", human_message_template)
    ])

    # Instantiate the parser
    svg_parser = SvgOutputParser()

    # Create chain
    chain = prompt | llm | svg_parser

    #invoke the chain
    response = chain.invoke({
        "svg_text": updated_svg_text,
        "analysis_report": analysis_report,
        "user_prompt": user_prompt
        })

    return response

In [289]:
user_prompt = "Animate the planes coming from each side and crossing over"
new_svg = animate_svg(updated_svg_text=updated_svg_text, analysis_report=response, user_prompt=user_prompt)

In [285]:
print(new_svg)

<svg xmlns="http://www.w3.org/2000/svg" height="48px" viewBox="0 -960 960 960" width="48px" fill="#e3e3e3">
  <path id="path_head" d="M480.08-814q-30.08 0-51.58-21.42t-21.5-51.5q0-30.08 21.42-51.58t51.5-21.5q30.08 0 51.58 21.42t21.5 51.5q0 30.08-21.42 51.58t-51.5 21.5Z">
    <animate attributeName="fill" values="#e3e3e3;red;#e3e3e3" dur="2s" repeatCount="indefinite"/>
  </path>
  <path id="path_body" d="M376-200v-494q-63-5-127-14.5T130-732l15-60q80 20 164.5 29t170.5 9q86 0 170.5-9T815-792l15 60q-55 14-119 23.5t-127 14.64V-200h-60v-244h-88v244h-60Z">
    <animate attributeName="fill" values="#e3e3e3;red;#e3e3e3" dur="2s" repeatCount="indefinite"/>
  </path>
  <path id="path_base_left" d="M317 0q-17 0-27.5-10.5T279-38q0-17 10.5-27.5T317-76q17 0 27.5 10.5T355-38q0 17-10.5 27.5T317 0Z">
    <animateTransform attributeName="transform" type="scale" values="1 1;1.2 1.2;1 1" dur="2s" repeatCount="indefinite" begin="0s" additive="sum" restart="always" keyTimes="0;0.5;1" origin="317 -38"/>
  </p

In [290]:
with open("/Users/vishwajeetsawant/Work/animater/output/coffee2.svg", "w", encoding="utf-8") as f:
    f.write(new_svg)
